# Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from math import sqrt
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Embedding, TimeDistributed, LeakyReLU
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot
from pickle import load
from numpy import *
from pandas import *

In [ ]:
!pip install -q -U keras-tuner
from kerastuner import HyperModel
from kerastuner.tuners import BayesianOptimization

     |████████████████████████████████| 71kB 9.7MB/s 


# Loading Train-Test data using NumPy

In [ ]:
X_train = np.load("X_train.npy", allow_pickle=True)
Y_train = np.load("y_train.npy", allow_pickle=True)
X_test = np.load("X_test.npy", allow_pickle=True)
Y_test = np.load("y_test.npy", allow_pickle=True)
Yc_train = np.load("yc_train.npy", allow_pickle=True)
Yc_test = np.load("yc_test.npy", allow_pickle=True)

# LSTM Model

In [ ]:
class LSTMHyperModel(HyperModel):
    
    def __init__(self, input_dim,feature_size,output_dim):
        self.input_dim = input_dim
        self.feature_size = feature_size
        self.output_dim = output_dim
        
    def build(self, hp):
        model = Sequential()
        model.add(
            Bidirectional(LSTM(
                units=hp.Int('units_directional', min_value = 32,max_value =512,step = 32),
                activation='tanh',
                input_shape=(input_dim,feature_size)
                )
            )
        )
        
        model.add(
            Dense(
                units=hp.Int('units_dense', min_value = 32,max_value =512,step = 32),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(Dense(1))
        
        model.compile(
            optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4,2e-2, 2e-3, 2e-4,3e-2, 3e-3, 3e-4,4e-2, 4e-3, 4e-4,5e-2, 5e-3, 5e-4])),loss='mse',metrics=['mse']
        )
        
        return model

In [ ]:
input_dim = X_train.shape[1]
feature_size = X_train.shape[2]
output_dim = Y_train.shape[1]
hypermodel = LSTMHyperModel(input_dim,feature_size,output_dim)

# Executing Bayesian Optimization

In [ ]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2
        )
tuner_bo.search(X_train, Y_train, epochs=150, validation_data=(X_test, Y_test), verbose=4)
best_model = tuner_bo.get_best_models(num_models=1)[0]
best_model.evaluate(X_test, Y_test)

Trial 10 Complete [00h 04m 50s]
val_mse: 1.3099586367607117

Best val_mse So Far: 0.0015383216086775064
Total elapsed time: 00h 44m 08s
INFO:tensorflow:Oracle triggered exit
51/51 [==============================] - 2s 4ms/step - loss: 0.0015 - mse: 0.0015


[0.0015233262674883008, 0.0015233262674883008]

# Results

In [ ]:
tuner_bo.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_mse', direction='min')
Trial summary
Hyperparameters:
units_directional: 448
units_dense: 64
dense_activation: tanh
learning_rate: 0.001
Score: 0.0015383216086775064
Trial summary
Hyperparameters:
units_directional: 448
units_dense: 192
dense_activation: sigmoid
learning_rate: 0.001
Score: 0.0015778450760990381
Trial summary
Hyperparameters:
units_directional: 480
units_dense: 32
dense_activation: sigmoid
learning_rate: 0.002
Score: 0.0016240469412878156
Trial summary
Hyperparameters:
units_directional: 352
units_dense: 64
dense_activation: tanh
learning_rate: 0.002
Score: 0.0022264543222263455
Trial summary
Hyperparameters:
units_directional: 512
units_dense: 32
dense_activation: relu
learning_rate: 0.0003
Score: 0.0092625527177006
Trial summary
Hyperparameters:
units_directional: 256
units_dense: 32
dense_activation: relu
learning_rate: 0.01
Score: 0.032957484014332294
Trial summary
Hyperparamete